In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.stats import binom
from scipy.stats import gamma
from matplotlib import pyplot as plt
from typing import Tuple, List, Callable, Any

from file_util import *

In [ ]:
# General settings
num_runs = 1
starting_seed = 0
seed_multiplier = 100

# Validator settings
num_nodes = 31 
num_consensus = 2000
base_time_limit = 10000
node_processing_distribution = "exp"
node_processing_parameters = [3]
consensus_protocol = "HS"

## Fault settings
num_faults = 0
fault_type = "UR"
fault_parameters = []

# Network settings
## Switch settings
switch_processing_distribution = "exp"
switch_processing_parameters = [0]
message_channel_success_rate = 1

network_type = "df"
network_parameters = []

In [ ]:
# More utility methods for analysis
def get_minima(series: pd.Series):
    return series[(series < series.shift(1)) & (series < series.shift(-1))].iloc[0]

def get_minima_index(series: pd.Series):
    return series[(series < series.shift(1)) & (series < series.shift(-1))].index[0]

In [ ]:
def get_topology_data(topo: str, protocol: str, name: str, topo_params: List[int]=None):
    return get_fn_data(lambda name: name.get_switch_params()[0], lambda json: json[TOTAL_TIME_KEY], name=name,
                       num_nodes=num_nodes, base_time_limit=base_time_limit, 
                       topology=topo, protocol=protocol, num_faults=num_faults, 
                       node_dist==node_processing_distribution,
                       topology_params=topo_params)

In [ ]:
ibft_series_fc_8_4 = get_topology_data("fc", "ibft", "ibft_fc_8_4", topo_params=[8, 4])
hs_series_fc_8_4 = get_topology_data("fc", "hs", "hs_fc_8_4", topo_params=[8, 4])
topo_df = pd.DataFrame([ibft_series_fc_8_4, hs_series_fc_8_4]).transpose()

topo_df.iloc[6:].plot(style=".-", grid=True, title="Folded clos, n=31, Half-radix = 4", xlabel="switch processing rate", ylabel="time to consensus", figsize=(10, 5))

In [ ]:
ibft_series_fc_3 = get_topology_data("df", "ibft", "ibft_df_3", topo_params=[3])
hs_series_fc_3 = get_topology_data("df", "hs", "hs_df_3", topo_params=[3])

topo_df = pd.DataFrame([ibft_series_fc_3, hs_series_fc_3]).transpose()

topo_df.iloc[6:].plot(style=".-", grid=True, title="Dragonfly, n=31, 3 switch per group, 4 terminals per switch", xlabel="switch processing rate", ylabel="average time to consensus", figsize=(10, 5))


In [ ]:
def ibft_fc_model(n, k, mu, _lambda):
    min_time = (2 * n + 1) / mu
    model_time = 2 * k * (2 * n - 1) / _lambda
    return max(min_time, model_time) 

def ibft_df_model(n, g, mu, _lambda):
    k = np.ceil(n / (g * (g + 1))) 
    min_time = (2 * n - 1) / mu
    model_time = 2 * k * (2 * n - 1 + 2 * k * (g * (g - 1))) / _lambda
    return max(min_time, model_time)

cross_topo_df = pd.DataFrame([ibft_series_fc_8_4, ibft_series_fc_3]).transpose()
cross_topo_df["fc_prediction"] = topo_df.index.map(lambda x : ibft_fc_model(num_nodes, 4, 3, x))
cross_topo_df["df_prediction"] = topo_df.index.map(lambda x : ibft_df_model(num_nodes, 3, 3, x))
cross_topo_df.iloc[6:].plot(style=".-", grid=True, title="IBFT Pred Test", figsize=(10, 5), xlabel="switch processing rate", ylabel="average time to consensus")

In [ ]:
def hs_pred_fn_2(n, mu, _lambda):
    f = (n - 1) // 3
    pred_1 = (n - 1) * 8 / _lambda
    pred_2 = (4 * (n - f)) / mu + 3 * (n) / _lambda + (n / 2 + 4.65) / _lambda 
    pred_3 = (4 * (n - f)) / mu + 3 * (f + 1) / mu + (n / 2 + 4.65) / _lambda
    # first term is processing delay per round, n - f at leader and 1 elsewhere
    # second term is remainder time that the leader clears up redundant extra messages vs time the first next-phase message takes to enter. Happens for 3 phases.
    # Last term is the average time taken for the final message to reach the desired node, with n/2 being half the nodes and 5.23 being the expected hop count
    return max(pred_1, pred_2, pred_3)

topo_df["hs_pred"] = topo_df.index.map(lambda x: hs_pred_fn_2(31, 3, x))
topo_df.iloc[4:][["hs_pred", "hs_fc_8_4"]].plot(style=".-", grid=True, title="HS Model", xlabel="switch processing rate", ylabel="time to consensus", figsize=(10,5))